# Double torus animation 

## YouTube video showing result

In [7]:
from IPython.display import HTML
html = '<iframe width="{width}" height="{height}" src="https://www.youtube.com/embed/{video_id}?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>'
HTML(html.format(video_id='3ueARDFk64U',width=800,height=500))

# Imports and utility functions

In [ ]:
import sys
sys.path.append("..")
from gblend.geometry import *
from gblend.levels import *
import graphics.d3 as g3d
from sympy import pi
import numpy as np
import plotly.graph_objects as go
import os
import shutil

camera_x = 1.0
camera_y = -1.5
camera_z = 2.0

hsd = 4 #
box = [-hsd,hsd,-hsd,hsd,-hsd,hsd]
rs = 120
res = [rs,rs,rs]

def cos_interp(a,b,cnt):
    ts = np.linspace(0,1,cnt)
    return a+(b-a)*(1-np.cos(np.pi*ts))/2.0

def interp(vs, cnt):
    fp = []
    for i in range(len(vs)-1):
        fp.append(cos_interp(vs[i],vs[i+1],cnt))
    fp = np.array(fp).flatten()
    x = np.linspace(1,cnt,cnt)
    xp = np.linspace(1,cnt,len(fp))
    return np.interp(x,xp,fp)
    

def create_movie_folder(movie_name, base_path = os.getenv("HOME") ):
    path = base_path+'/'+movie_name
    if os.path.exists(path):
        print(path + " exists")
        ans = 'y' #input("remove (y/n)") 
        if ans == 'y': 
            print("removing "+ path)
            try:
                shutil.rmtree(path)
            except OSError:
                print("Error: %s : %s" % (dir_path, e.strerror))
            else:
                print(path+" succesfully removed")
        
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)
        return path
    return path

def save_geometry(obj, frame_no, i, camera=None):
    if camera is None:
        camera = dict(up=dict(x=0, y=0, z=1),
                      center=dict(x=0, y=0, z=0),
                      eye=dict(x=camera_x, y=camera_y, z=camera_z))
    ps, ts, ns, vals = geometry_level(obj,box,res)
    fig = g3d.get_figure(g3d.mesh3d(ps, ts), width=800, height=600,camera=camera,layout_no=0)
    frame_name = 'mov'+'{:>04}'.format(str(frame_no))+ext
    file_path = movie_path+'/'+frame_name
    fig.write_image(file_path)

# Create folder to store animation frames

In [2]:
movie_name = 'DoubleTorus'
movie_path = create_movie_folder(movie_name)
ext = '.png'

/home/valeroc/DoubleTorus exists
removing /home/valeroc/DoubleTorus
/home/valeroc/DoubleTorus succesfully removed
Successfully created the directory /home/valeroc/DoubleTorus 


# Rendering animation frames as .png files

In [3]:
b1_0 = Ball3D()
b2_0 = Ball3D()
c1_0 = Cylinder3D()
c2_0 = Cylinder3D()


fns = ["Double Torus Base", "Making Holes", "Twist Handle", "350 View", "Camera Adjust", "Quadratic Displacement"]
fcs = np.array([100,100,100,200,100,150]) # Frames count per step
fcc = [0] # Cummulative framse count
fcc.extend(np.cumsum(fcs)[0:len(fcs)-1])

def header(step):
    print("-------------------------------------------------")
    print("("+str(step+1)+" of " + str(len(fns))+")")
    print(fns[step]+": Rendering "+ str(fcs[step])+ " frames")
    
    
def progress(i,total):
    percent = str(int(float(i/(total-1))*100))+'%'
    s = "Progress: " + percent + '\r'
    print(s,end='')
    
def step0():
    global b1,b2
    step = 0
    header(step)
    fc = fcs[step]
    sc = interp([1.0,2.0],fc) # scales
    tx = interp([2.0,1.8],fc) # translation in x-direction
    for i in range(0,fcs[step]):
        b1 = b1_0.scaled(sc[i],sc[i],1).translated(-tx[i],0,0)
        b2 = b2_0.scaled(sc[i],sc[i],1).translated(tx[i],0,0)
        fn = fcc[step]+i
        save_geometry(b1 | b2,fn,i)
        progress(i,fc)
        
        
    
def step1():
    global c1, c2
    step = 1
    header(step)
    fc = fcs[step]
    sc = interp([0.2,0.5],fc) 
    for i in range(0,fc):
        c1 = c1_0.scaled(sc[i],sc[i],1).translated(-1.8,0,0)
        c2 = c1_0.scaled(sc[i],sc[i],1).translated(1.8,0,0)
        obj = (b1-c1) | (b2-c2)
        fn = fcc[step]+i
        save_geometry(obj,fn,i)
        progress(i,fc)
    
def step2():
    global b2r, c2r, obj
    step = 2
    header(step)
    fc = fcs[step]
    ts = interp([0.0,-np.pi/4],fc) 
    for i in range(0,fc):
        b2r = b2.rotated(1,0,0,ts[i])
        c2r = c2.rotated(1,0,0,ts[i])
        obj = (b1-c1) | (b2r-c2r)
        fn = fcc[step]+i
        save_geometry(obj,fn,i)
        progress(i,fc)
        
def step3():
    step = 3
    header(step)
    fc = fcs[step]
    ts = interp([0.0,2*np.pi],fc) 
    for i in range(0,fc):
        th =  ts[i]
        camera_xn = camera_x*np.cos(th)-camera_y*np.sin(th)
        camera_yn = camera_x*np.sin(th)+camera_y*np.cos(th)
        camera = dict(up=dict(x=0, y=0, z=1),
                      center=dict(x=0, y=0, z=0),
                      eye=dict(x=camera_xn, y=camera_yn, z=camera_z))
        
        fn = fcc[step]+i
        save_geometry(obj,fn,i, camera=camera)
        progress(i,fc)

def step4():
    global camera_y, camera_z
    step = 4
    header(step)
    fc = fcs[step]
    ts = interp([0,np.pi-np.arctan2(camera_z,camera_y)],fc)
    for i in range(0,fc):
        th =  ts[i]
        camera_yn = camera_y*np.cos(th)-camera_z*np.sin(th)
        camera_zn = camera_y*np.sin(th)+camera_z*np.cos(th)
        camera = dict(up=dict(x=0, y=0, z=1),
                      center=dict(x=0, y=0, z=0),
                      eye=dict(x=camera_x, y=camera_yn, z=camera_zn))
        fn = fcc[step]+i
        save_geometry(obj,fn,i, camera=camera)
        progress(i,fc)
    camera_y = camera_yn
    camera_z = camera_zn
        
        
def step5():
    step = 5
    header(step)
    fc = fcs[step]
    ts = interp([0,.15],fc)
    for i in range(0,fc):
        fn = fcc[step]+i
        save_geometry(obj.displaced(ts[i]*(x**2-y**2)),fn,i)
        progress(i,fc)

#
# Rendering
#
steps = [step0,step1,step2,step3,step4,step5]
def do_steps(step_ids):
    for i in step_ids:
        steps[i]()
        
do_steps([0,1,2,3,4,5])

-------------------------------------------------
(1 of 6)
Double Torus Base: Rendering 100 frames
-------------------------------------------------
(2 of 6)
Making Holes: Rendering 100 frames
-------------------------------------------------
(3 of 6)
Twist Handle: Rendering 100 frames
-------------------------------------------------
(4 of 6)
350 View: Rendering 200 frames
-------------------------------------------------
(5 of 6)
Camera Adjust: Rendering 100 frames
-------------------------------------------------
(6 of 6)
Quadratic Displacement: Rendering 150 frames
